In [49]:
from IPython.core.display import HTML
HTML("""
<style>
.container { width:100% !important; }
</style>
""")

#### Imports

In [50]:
import numpy as np

### Inputs

In [51]:
s = 100
K = 105
sigma = 0.2
r = 0.05
T = 1
n = 100
kind = 'call'
style = 'European'

### Binomial Tree Model

In [52]:
def intrinsic_value(S,K,kind):
    assert kind in ['call','put'],"Must be call or put"
    return max(S-K,0) if kind == 'call' else max(K-S,0)
    
def binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European'):
    assert kind in ['call','put'],"Must be call or put"
    assert style in ['European','American'],"Must be European or American"
    
    h = T/n
    u = np.exp(sigma*np.sqrt(h))
    d = 1/u
    p_rn = (np.exp(r*h)-d)/(u-d)
    discount_factor = np.exp(-r*h)
    
    V = np.zeros(shape=(n+1,n+1))
    
    # At expiry (j=n) we know the option values
    for i in range(n+1):
        V[i,n]=intrinsic_value(S*u**i*d**(n-i),K,kind)
    # Go backwords in time
    for j in range(n-1,-1,-1):
        for i in range(j+1):
            continuation_value=discount_factor*(p_rn*V[i+1,j+1]+(1-p_rn)*V[i,j+1])
            if style == 'American':
                early_exercise_value = intrinsic_value(S*u**i*d**(j-i),K,kind=kind)
                V[i,j]=max(early_exercise_value,continuation_value)
            else:
                V[i,j]=continuation_value
    return V[0,0]
            
    
        
        
    
    

In [53]:
binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European')

8.026229025058432

In [54]:
binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='American')

8.026229025058432

In [55]:
binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='American')

8.747558705041387

In [56]:
binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='European')

7.90531859763425

## Black Scholes Formula

**Black-Scholes Formula**

The Black-Scholes formula is a cornerstone in options pricing, providing a theoretical value for European-style options. It relies on several key assumptions about market behavior and the underlying asset.

**Mathematical Representation**

$$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$$

$$
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
$$

where:

*   $C$: Call option price
*   $P$: Put option price
*   $S_0$: Current price of the underlying asset
*   $K$: Strike price of the option
*   $r$: Risk-free interest rate
*   $T$: Time to expiration (in years)
*   $N(x)$: Cumulative standard normal distribution function
*   $d_1 = \frac{\ln(\frac{S_0}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$
*   $d_2 = d_1 - \sigma \sqrt{T}$
*   $\sigma$: Volatility of the underlying asset's returns

**Interpretation**

*   The formula calculates the fair price of a call or put option based on the interplay of factors like the current stock price, strike price, time to expiration, risk-free rate, and volatility. 
*   $N(d_1)$ and $N(d_2)$ represent the probabilities that the option will expire in-the-money under the risk-neutral measure.

**Assumptions**

*   The underlying asset follows a geometric Brownian motion (constant volatility).
*   No dividends are paid on the underlying asset during the option's life.
*   No transaction costs or taxes.
*   Markets are frictionless (no arbitrage opportunities).
*   The risk-free interest rate is constant.

**Caveats**

While widely used, the Black-Scholes model has limitations. Real-world markets often exhibit features that violate its assumptions, such as jumps in prices or changing volatility. Therefore, it's crucial to understand its limitations and consider more sophisticated models when dealing with complex market dynamics.

Let me know if you'd like to delve deeper into any specific aspect or explore extensions to the Black-Scholes model! 
            

In [72]:
from scipy.stats import norm 
N = norm.cdf

def bs_price(S,K,sigma,r,T,n,kind='call'):
    assert kind in ['call','put'],"Must be call or put"
    d1 = (np.log(S/K)+(r+0.5*sigma**2)*T)/sigma/np.sqrt(T)
    d2 = d1-sigma*np.sqrt(T)
    return S*N(d1)-K*np.exp(-r*T)*N(d2) if kind=='call' else K*np.exp(-r*T)*N(-d2)-S*N(-d1)





In [73]:
bs_call_price=bs_price(S,K,sigma,r,T,n,kind='call')
bs_put_price=bs_price(S,K,sigma,r,T,n,kind='put')

In [74]:
binomial_call_prices={}
for n in [100,200,500,1000,10000]:
    binomial_call_prices[n]=binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European')

In [81]:
{k:np.round(v,4) for k,v in binomial_call_prices.items()},np.round(bs_call_price,4)

({100: 8.0262, 200: 8.026, 500: 8.0232, 1000: 8.0211, 10000: 8.0214}, 8.0214)

In [83]:
binomial_put_prices={}
for n in [100,200,500,1000,10000]:
    binomial_put_prices[n]=binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='European')

In [84]:
{k:np.round(v,4) for k,v in binomial_put_prices.items()},np.round(bs_put_price,4)

({100: 7.9053, 200: 7.9051, 500: 7.9023, 1000: 7.9001, 10000: 7.9005}, 7.9004)